# Baseline

In [ ]:
from darts.datasets import AirPassengersDataset
series = AirPassengersDataset().load()
series

In [1]:
from darts.datasets import AirPassengersDataset
from darts.models import ARIMA
from darts.utils.timeseries_generation import datetime_attribute_timeseries
series = AirPassengersDataset().load()
# optionally, use some future covariates; e.g. the value of the month encoded as a sine and cosine series
future_cov = datetime_attribute_timeseries(series, "month", cyclic=True, add_length=6)
# define ARIMA parameters
model = ARIMA(p=12, d=1, q=2)
model.fit(series, future_covariates=future_cov)
pred = model.predict(6, future_covariates=future_cov)
pred.values()

ModuleNotFoundError: No module named 'darts'